In [5]:
import sympy as sp
import numpy as np
import galois as gf
import pandoc
import math
np.set_printoptions(legacy='1.25')

## Problem 1

Consider the matrix

$ A = \begin{bmatrix}
3 & 1 & 6 \\
3 & 0 & 4 \\
1 & 3 & 2 
\end{bmatrix}  $

in GF(7).

(a) Find the determinant of $A$. Is it invertible?

>A matrix is invertible if it is a square $n$ by $n$ matrix, and the determinant is non-zero.

In [201]:
# det| a b | = a*e - b*d
#    | d e |

#matrix: numpy array.
#p: galois field value GF(p)
def determinant_2x2_GF(matrix,p) :
    return np.mod(int(np.linalg.det(matrix)),p)

#matrix: numpy array.
#p: galois field value GF(p)
def determinant_3x3_GF(matrix,p) :
    return np.mod(int(np.linalg.det(matrix)),p)

In [202]:
# A = np.array([[0,4],[3,2]])
# print(f'Testing determinant_2x2_GF...')
# print(f'Expected value = 2. Answer = {determinant_2x2_GF(A,7)}')

A = np.array([[3,1,6],[3,0,4],[1,3,2]])

print(f'The determinant of A = {determinant_3x3_GF(A,7)}')

The determinant of A = 2


In [203]:
print(len(A))

3


> The determinant of A = 2, which is not zero, therefore A is invertible.

> I used numpy's numpy.linalg.det() function which returns the determinant of a matrix. I take the modulus of the result.

(b) Recall that in real numbers, the inverse of $A$ can be found as

$ A^{-1} = \frac{1}{det(A)} \begin{bmatrix}
det(A(1, 1)) & − det(A(1, 2)) & det(A(1, 3)) \\
− det(A(2, 1)) & det(A(2, 2)) & − det(A(2, 3)) \\
det(A(3, 1)) & − det(A(3, 2)) & det(A(3, 3)) 
\end{bmatrix}  $

where $A(i, j)$ is the matrix obtained by removing row $i$ and column $j$ of matrix $A$. For instance,

$ A(1,2) = \begin{bmatrix}
3 & 4 \\
1 & 2 
\end{bmatrix}  $

Note that $±$ det($A(i, j)$) is still an integer, and hence, you can find the inverse of $A$ in GF(7) by replacing division by det($A$) by multiplying by in the multiplicative inverse of det($A$) in GF(7). Now, use the method above to find the inverse of $A$ in GF(7). You may use MATLAB or any other software. Describe your method.

In [277]:
#Finds inverse in modulo. e.g. 2 inverse in mod 3 is 2 b/c 2*2 = 4(mod 3) = 1
#==Inputs== 
#input: value to find the inverse of
#modulo: value of the modulus
def find_modulo_inverse(input, modulo) :
    i=modulo
    while i>0 :
        if np.mod(input*i,modulo) == 1 :
            return i
        i-=1
    return 0

def modulo_divide(a, b, m):
    a = np.mod(a,m)
    inv = int(find_modulo_inverse(b,m))
    if inv == 0 :
        print("Division not Defined")
        return 0
    else :
        return np.mod(inv*a,m)


#matrix: nxn numpy array.
#p: p in GF(p)
#returns the inversem matrix.
def inverse_matrix_GF(matrix, p) :
    determinant_inv = find_modulo_inverse(int(np.mod(np.linalg.det(matrix),p)),p) # finds inverse of the determinant, to be multiplied into matrix later.
    output = np.zeros([len(matrix),len(matrix)],dtype=int)
    i=0
    negative = 1 # +/- 1 depending on the row/column index.
    while i<len(matrix):
        j=0
        while j<len(output) :
            #print(f'det_inv = {determinant_inv}')
            #print(f'matrix[i={i}][j={j}] = {matrix[i][j]}')
            #print(f'output[i,j] = {matrix[i][j]*determinant_inv*negative*int(np.linalg.det(np.delete((np.delete(matrix, i,axis=0)),j,axis=1)))}')
            output[i,j] = np.mod((matrix[i][j]*determinant_inv*negative*int(np.linalg.det(np.delete((np.delete(matrix, i,axis=0)),j,axis=1)))),p) #deletes row i and column j, and finds the determinant of the resulting matrix, and divides the determinant into the result.
            negative = negative*-1 #alternates the negative sign to the matrix
            j+=1
        i+=1
    return output

In [286]:
A = np.array([[3,1,6],[3,0,4],[1,3,2]])
A_inv = inverse_matrix_GF(A,7)
print('A^-1 = ')
print(A_inv)

print('')
#Testing to see if inverse produced identity matrix.
print('A*A^-1 = ')
print(np.matmul(A,A_inv))


A^-1 = 
[[3 6 6]
 [3 0 5]
 [2 2 4]]

A*A^-1 = 
[[24 30 47]
 [17 26 34]
 [16 10 29]]


> $ A^{-1} = \begin{bmatrix}
3 & 6 & 6 \\
3 & 0 & 5 \\
2 & 2 & 4 
\end{bmatrix}  $

> $A \cdot A^{-1} = \begin{bmatrix}
24 & 30 & 47 \\
17 & 26 & 34 \\
16 & 10 & 29 
\end{bmatrix} = 
\begin{bmatrix}
3 & 2 & 5 \\
3 & 5 & 6 \\
2 & 3 & 1 
\end{bmatrix}$

Using a RREF calculator, https://www.emathhelp.net/en/calculators/linear-algebra/reduced-row-echelon-form-rref-calculator/?i=%5B%5B3%2C2%2C5%5D%2C%5B3%2C5%2C6%5D%2C%5B2%2C3%2C1%5D%5D&reduced=on ,

we get:

$A \cdot A^{-1} = \begin{bmatrix}
1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 1 
\end{bmatrix}$


## Problem 2

Consider a general block cipher algorithm that maps $n$ bits to $n$ bits.

(a) How many different such mapping (including the identity map) exist?

> $2^{N}!$

(b) Let $K(n)$ be your answer to part (a). Bob and Alice want to communicate privately. In order to set the mapping, they shared a long table including all possible $K(n)$ mappings and associated a number between $0$ and $K(n) − 1$ to each. Then, they wrote the corresponding number for each mapping in Binary representation. For instance, if $K(n) = 1000$, mapping number $347$ is represented by $’0101011011’$. Then, each time Bob wants to send a private message to Alice, he chooses one of these mappings and sends her its binary representation through a secure channel. How many bits does Bob have to send for each mapping?

In [261]:
for n in range(0,6) :
    print(f'n={n}, (2^{n})! = {math.factorial(2**n)}')

n=0, (2^0)! = 1
n=1, (2^1)! = 2
n=2, (2^2)! = 24
n=3, (2^3)! = 40320
n=4, (2^4)! = 20922789888000
n=5, (2^5)! = 263130836933693530167218012160000000


> 3 bits.


(c) Recall that we mentioned in class that $n$-to-$n$ bits mappings can be also represented by $n$ × $2n$ bits (similar to Table. 4.1 of the textbook). Is this number less or greater than what you found in part (b)? Explain the discrepancy.

> This number would be greater, because for each possible mapping, there are twice as many choices to choose from.

## Problem 3

Consider the GF($2^{8}$) generated by $q(x) = x^{8} + x^{7} + x^{2} + x + 1$. Evaluate the following quantities:

(a) $\{BC\} + \{2E\}$.

In [63]:
bin(0xBC)+bin(0x2E)


def hex_add(A,B,p) :
    A_array = [x for x in bin(A)]
    B_array = [x for x in bin(B)]
    A_array.pop(0) #get rid of '0b' from the arrays
    A_array.pop(0)
    B_array.pop(0)
    B_array.pop(0)
    A_array = [int(x) for x in A_array] #convert each element to integer
    B_array = [int(x) for x in B_array]


    while len(A_array) < 8 : #add zeros to make array an 8 bit number
        A_array.insert(0,0)
    while len(B_array) < 8 :
        B_array.insert(0,0)

    output = [0] * 8
    i=0
    while i<8 :
        output[i] = (A_array[i] + B_array[i])%p #add each bit, modulus 2
        i+=1
    return hex(int('0b' + ''.join(str(x) for x in output), 2))

print(hex_add(0xBC,0x2E,2))


0x92


> $\{BC\} + \{2E\} = \{92\}$

(b) $\{0A\} \cdot \{A3\}$.

In [182]:
#Finds the degree of a polynomial vector by ignoring finding the first non-zero value.
def find_degree(polynomial) :
    shift = 0
    for i in polynomial :
        if i!=0 :
            return len(polynomial)-shift-1
        else : 
            shift +=1
    return 0

#Removes all zeros in a polynomial vector preceeding the first non-zero element.
#e.g. [0, 0, 0, 2, 0, 3, 0] --> [2, 0, 3, 0]
def remove_zeros(polynomial) :
    shift = 0
    for i in polynomial :
        if i!=0 :
            while shift>0 :
                polynomial.pop(0)
                shift-=1
            break
        else : 
            shift +=1
    return polynomial


# Add A and B modulo p.
#==INPUTS==
#prime: prime number from p in GF(p)
#A: an array of polynomial coefficients e.g. 2x^2 + 1 --> [2 0 1]
#B: same formate as A
#M: do not care
#p: p in GF(p), prime number
#==OUTPUTS==
#Returns polynomial coefficient array that is 
def MyGFAdd(A, B, M, p) :
    degree_A = find_degree(A) #highest degree of A
    degree_B = find_degree(B) #highest degree of B
    degree_diff = abs(degree_A - degree_B)
    A = remove_zeros(A) #remove preceeding zeros
    B = remove_zeros(B)

    #Add A and B differently depending on which one is longer or shorter.
    if degree_A > degree_B : 
        #print('A > B')
        output = [0] * len(A) # A is a longer array
        i=0
        while i<len(A) :
            if i < degree_diff : output[i] = A[i] #don't start adding B until A and B are at the same degree.
            else : output[i] = A[i]+B[i-degree_diff]
            i+=1

    else :
        if degree_A < degree_B : 
            #print('A < B')
            output = [0] * len(B) # B is a logner array
            i=0
            while i<len(B) :
                if i < degree_diff : output[i] = B[i] #don't start adding B until A and B are at the same degree.
                else : output[i] = B[i]+A[i-degree_diff]
                i+=1
        else : #degree_A = degree_B
            #print('A=B')
            output = [0] * len(A)
            i=0
            for element in A :
                output[i] = element+B[i]
                i+=1
    
    # apply modulus to output.
    i=0
    while i<len(output) :
        output[i] = np.mod(output[i], p)
        i+=1

    return output

#Multiplies one value into a full polynomial
#==INPUTS==
#k: scalar coefficent of single value.
#degree_k: degree of 
#A: polynomial coefficient vector.
#==OUTPUTS==
#Returns a polynomial coefficient array: A*kx^(degree)
def single_mult(k,degree_k,A) :
    degree_A = find_degree(A)
    A = remove_zeros(A)
    output = [0] * (degree_A+degree_k+1)

    if k == 0 :
        return output

    i=0
    while i<len(A):
        output[i] = k*A[i]
        i+=1

    return output

#Multiplies two polynomials, ignoring degree > M
def raw_poly_mult_GF(A, B, GF) :
    degree_A = find_degree(A)
    degree_B = find_degree(B)
    
    if degree_A > degree_B : 
        #print('A > B')
        partial_output = [[0 for _ in range(degree_A+degree_B+1)] for _ in range(len(B))]
        i=0
        for element in B :
                partial_output[i] = single_mult(element,(degree_B-i),A)
                i+=1

    else :
        if degree_A < degree_B : 
            #print('A < B')
            partial_output = [[0 for _ in range(degree_A+degree_B+1)] for _ in range(len(A))]
            i=0
            for element in A :
                partial_output[i] = single_mult(element,(degree_A-i),B)
                i+=1

        else : #degree_A = degree_B
            #print('A=B')
            partial_output = [[0 for _ in range(degree_A+degree_B+1)] for _ in range(len(A))]
            i=0
            for element in A :
                partial_output[i] = single_mult(element,(degree_A-i),B)
                i+=1


    max_degree = 0
    for element in partial_output :
        deg = find_degree(element)
        if deg > max_degree :
            max_degree = deg
    #print(f'max_deg={max_degree}')

    output = [0] * (max_degree+1)

    output = partial_output[0]
    #print(f'partial_output={partial_output}')
    i=1
    while i<(len(partial_output)) :
        output = MyGFAdd(output,partial_output[i],0,GF)
        #print(f'output={output}')
        i+=1


    return output

#Finds what the highest degree coefficient is equal to. Assumes that that coefficient = 1.
#e.g. 
# input: [1, 1, 1] output: [2, 2]
#      (x^2 + x + 1)   =   (2x+2)
def find_highest_coefficient(M,p) :
    M[0] = 0
    M = list(np.multiply(M,(-1)))
    M = np.mod(remove_zeros(M),p)
    return M

#Returns the multiplication of A and B in GF(p^k)
#==INPUTS==
#A: coefficient polynomial array A
#B: coefficient polynomial array B
#M: m(x) is an irreducible polynomial in GF(p^k)
#p: prime number p in GF(p)
#==OUTPUTS==
#Returns a polynomial coefficient array of the form Poly [c_n, c_n-1, ..., c1 , c0] --> Poly = c_n*x^n + c_(n-1)*x^(n-1) + ... c1*x + c0
def MyGFMult(A, B, M, p) :
    #find the degree of each polynomial
    A_array = [x for x in bin(A)]
    B_array = [x for x in bin(B)]
    A_array.pop(0) #get rid of '0b' from the arrays
    A_array.pop(0)
    B_array.pop(0)
    B_array.pop(0)
    A_array = [int(x) for x in A_array] #convert each element to integer
    B_array = [int(x) for x in B_array]

    degree_A = find_degree(A_array)
    degree_B = find_degree(B_array)
    degree_M = find_degree(M)
    degree_diff = abs(degree_A - degree_B)
    #get rid of any preceeding zeros
    A = remove_zeros(A_array)
    B = remove_zeros(B_array)
    M = remove_zeros(M)

    # print(f'A={A}, degree={degree_A}')
    # print(f'B={B}, degree={degree_B}')
    # print(f'M={M}, degree={degree_M}')

    #Multiply A and B, still including terms with degree > degree_M
    raw_mult = raw_poly_mult_GF(A,B,p)
    #print(f'rawmult={raw_mult}')
    raw_degree = find_degree(raw_mult)
    inv_M = find_highest_coefficient(M,p) # not actually an inverse, but rather what the highest degree of M equals. e.g. M == x^2 + x + 1, inv_M = 2x+2
    
    #print(f'raw_degree = {raw_degree}')
    #if there needs to be substitutions.
    #e.g. for GF(3), M == x^2 + x + 1, (x^2 = 2x+2), and raw_mult == x^3 --> raw_mult = x(x^2) = x(2x+2) = 2x^2 + 2x = 4x + 4 + 2x = 6x + 4 = 1 (mod 3)
    if raw_degree >= degree_M : 
        degree_diff = raw_degree - degree_diff
        i=0
        while degree_diff>=0 :
            # print(f'degree_diff = {degree_diff}; degree_M = {degree_M}')
            # print(f'Raw_mult1={raw_mult}')
            # print(f'(len(raw_mult)-len(M)+1) = {(len(raw_mult)-len(M)+1)}')
            single_elem = [0] *(len(raw_mult)-len(M)+1)
            single_elem[0] = raw_mult[0]

            raw_mult[0] = 0 #remove highest degree of raw_mult for substitution

            #Debug print statements
            # print(f'Raw_mult2={raw_mult}')
            #print(f'i={i}')
            # print(f'single_elem={single_elem}')
            # print(f'inv_m={inv_M}')
            # print(f'raw_poly_mult_GF(inv_M,single_elem,p)={raw_poly_mult_GF(inv_M,single_elem,p)}')

            #(single_elem*inv_M) is the replacement for the highest degree power of raw_mult, whose result
            #is added back to raw_mult to fully apply the subsitution
            raw_mult = MyGFAdd(raw_poly_mult_GF(inv_M,single_elem,p),raw_mult,M,p) #
            raw_mult = remove_zeros(raw_mult) #important for reducing len(raw_mult) to proper size.
            
            raw_degree = find_degree(raw_mult)
            degree_diff = raw_degree - degree_M
            # print(f'degree_diff = {degree_diff}')

            #print(f'Raw_mult3={raw_mult}')
            # print('')
            
            i+=1

    return raw_mult

In [198]:
Mx = [1,1,0,0,0,0,1,1,1]
p = 2
print('Answer = ')
print(hex(int('0b'+''.join([str(x) for x in MyGFMult(0x0A,0xA3,Mx, p)]),2)))

Answer = 
0xcb


> $\{0A\} \cdot \{A3\} = \{CB\}$

In [229]:
Mx2 = [1,0,0,0,1,1,0,1,1]
p = 2
MyGFMult(0x03,0x6E,Mx2, p)
#Checking with a known answer...

[1, 0, 1, 1, 0, 0, 1, 0]

(c) $\{84\}−1$

> $\{84\} - 1 = \{83\}$

## Problem 4

Consider GF($3^{3}$) generated by $m(x) = x^{3} + 2x^{2} + 1$, and let $\alpha$ be a root of $m(x)$. Let $a = 2\alpha^{2} + \alpha + 1$. We have already seen that we can find the multiplicative inverse of $a$ as follows: Let $b = b_{2}\alpha^{2} + b_{1}\alpha + b_{0}$ is $a^{−1}$. Then, we there should exists some $q$ such that

$a \cdot b = m(α) \cdot q + 1.$

(a) We can argue that $q$ should be of the form of $q = q_{1}\alpha + q_{0}$. Why $q$ does not have any term of $\alpha^{2}$?

>Because then $m(\alpha)\cdot q$ would be of the form: $c_{4} \alpha ^{4} + c_{3}\alpha ^{3} + c_{2}\alpha ^{2} c_{1} \alpha + c_{0}$ which is a higher degree than $a \cdot b$

Now, to solve for (1), we can write

$a \cdot b = (2\alpha^{2} + \alpha + 1) \cdot (b_{2}\alpha^{2} + b_{1}\alpha + b_{0}) = (2b_{2})\alpha^{4} + (2b_{1} + b_{2})\alpha^{3} + (2b_{0} + b_{1} + b_{0})\alpha^{2} + (b_{1} + b_{0})\alpha + b_{0}$

$m(α) \cdot q + 1 = q_{1}\alpha^{4} + (2q_{1} + q_{0})\alpha^{3} + (2q_{0})\alpha^{2} + q_{1}\alpha + (q_{0}) + 1.$

So, we need to determine $(b_{2}, b_{1}, b_{0})$ and $(q_{1}, q_{0})$ such that

$2b_{2} = q_{1}$

$2b_{1} + b_{2} = 2q_{1} + q_{0}$

$2b_{0} + b_{1} + b_{0} = 2q_{0}$

$b_{1} + b_{0} = q_{1}$

$b_{0} = q_{0} + 1.$

(b) Write the equations in (2) as a linear system, i.e, $A \cdot x = b$, where $x = [b_{0}, b_{1}, b_{2}, q_{0}, q_{1}]^{T}$ . Find $A$ and $b$.

> $ A = \begin{bmatrix}
0 & 0 & 2 & 0 & 2 \\
0 & 2 & 1 & 2 & 1 \\
2 & 1 & 1 & 1 & 0 \\
1 & 1 & 0 & 0 & 2 \\
1 & 0 & 0 & 2 & 0 \\
\end{bmatrix}
;   b = \begin{bmatrix}
0 \\
0 \\
0 \\
0 \\
1
\end{bmatrix}
$

In [288]:
A = [[0,0,2,0,2],[0,2,1,2,1],[2,1,1,1,0],[1,1,0,0,2],[1,0,0,2,0]]
b = [[0],[0],[0],[0],[1]]

(c) Use Problem 1 to find the inverse of $A$ and solve the system for $x$. What is $b$?

In [292]:
inv_a = inverse_matrix_GF(A, 3)

In [293]:
print('A^-1 = ')
print(inv_a)
print('')
print('A*A^-1 = ')
print(np.matmul(A,inv_a))

A^-1 = 
[[0 0 1 0 0]
 [0 0 2 0 1]
 [0 1 0 2 0]
 [1 0 0 0 0]
 [1 0 0 2 0]]

A*A^-1 = 
[[2 2 0 8 0]
 [3 1 4 4 2]
 [1 1 4 2 1]
 [2 0 3 4 1]
 [2 0 1 0 0]]


> $ A^{-1} = \begin{bmatrix}
0 & 0 & 1 & 0 & 0 \\
0 & 0 & 2 & 0 & 1 \\
0 & 1 & 0 & 2 & 0 \\
1 & 0 & 0 & 0 & 0 \\
1 & 0 & 0 & 2 & 0 \\
\end{bmatrix} $

> $ A \cdot A^{-1} = \begin{bmatrix}
2 & 2 & 0 & 8 & 0 \\
3 & 1 & 4 & 4 & 2 \\
1 & 1 & 4 & 2 & 1 \\
2 & 0 & 3 & 4 & 1 \\
2 & 0 & 1 & 0 & 0 \\
\end{bmatrix} =
\begin{bmatrix}
2 & 2 & 0 & 2 & 0 \\
0 & 1 & 1 & 1 & 2 \\
1 & 1 & 1 & 2 & 1 \\
2 & 0 & 0 & 1 & 1 \\
2 & 0 & 1 & 0 & 0 \\
\end{bmatrix}
 \overset{RREF}{\rightarrow}
\begin{bmatrix}
1 & 0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 1 \\
\end{bmatrix}$

https://www.emathhelp.net/en/calculators/linear-algebra/reduced-row-echelon-form-rref-calculator/?i=%5B%5B2%2C2%2C0%2C2%2C0%5D%2C%5B0%2C1%2C1%2C1%2C2%5D%2C%5B1%2C1%2C1%2C2%2C1%5D%2C%5B2%2C0%2C0%2C1%2C1%5D%2C%5B2%2C0%2C1%2C0%2C0%5D%5D&reduced=on

(d) Write a computer program that computes the multiplicative inverse in a general finite field. Consider GF($p^{k}$) generated by $m(x) = m_{k}x^{k} + · · · + m_{1}x + m_{0}$ which is represented as $M = [m_{k}, m_{k−1}, . . . , m_{1}, m_{0}]$. For a value $a = a_{k−1}\alpha^{k−1} + · · · + a_{1}\alpha + a_{0}$, which is represented as $A = [a_{k−1}, . . . , a_{1}, a_{0}]$, compute $b = b_{k−1}\alpha^{k−1} + · · · + b_{1}\alpha + b_{0}$ such that $a \cdot b = 1$ and return $B = [b_{k−1}, . . . , b_{1}, b_{0}]$.

## Problem 5

Recall the S-box used in the Advanced Encryption Standard (AES). It maps a pair $\{xy\}$ to $\{x^{′}y^{′}\}$ = $S(\{xy\})$ according to the following steps:

• You first find {uv} = {xy}−1.

• Write {uv} in binary {uv} = [b7, . . . , b0].

• Compute

$ \begin{bmatrix}
b_{0}^{'} \\
b_{1}^{'} \\
b_{2}^{'} \\
b_{3}^{'} \\
b_{4}^{'} \\
b_{0}^{'} \\
b_{0}^{'} \\
b_{0}^{'} \\
\end{bmatrix} = \begin{bmatrix}
1 & 0 & 0 & 0 & 1 & 1 & 1 & 1 \\
1 & 1 & 0 & 0 & 0 & 1 & 1 & 1 \\
1 & 1 & 1 & 0 & 0 & 0 & 1 & 1 \\
1 & 1 & 1 & 1 & 0 & 0 & 0 & 1 \\
1 & 1 & 1 & 1 & 1 & 0 & 0 & 0 \\
0 & 1 & 1 & 1 & 1 & 1 & 0 & 0 \\
0 & 0 & 1 & 1 & 1 & 1 & 1 & 0 \\
0 & 0 & 0 & 1 & 1 & 1 & 1 & 1 
\end{bmatrix} \cdot
\begin{bmatrix}
b_{0} \\
b_{1} \\
b_{2} \\
b_{3} \\
b_{4} \\
b_{5} \\
b_{6} \\
b_{7} 
\end{bmatrix} +
\begin{bmatrix}
1 \\
1 \\
0 \\
0 \\
0 \\
1 \\
1 \\
0 
\end{bmatrix}$ (mod 2)

• Compute $\{x^{′}y^{′}\} = [b^{′}_{7}, . . . , b^{′}_{0}]$.

Write a computer program to generate the S-box. It should take $\{xy\}$ and return $\{x^{′}y^{′}\}$.